In [2]:
!pip install rapidfuzz
!pip install requests beautifulsoup4 rapidfuzz


In [6]:

!pip install requests beautifulsoup4 rapidfuzz

import requests
from bs4 import BeautifulSoup
from rapidfuzz import fuzz, process
import urllib3
import time
import re

# Data scraping of (https://tenymalagasy.org)

In [116]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print("=" * 60)
print("Scraping Complete Malagasy Dictionary")
print("=" * 60)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

# Step 1: Get all range links
base_url = "https://tenymalagasy.org"
main_url = "https://tenymalagasy.org/bins/alphaLists?lang=mg"

print(f"\nStep 1: Finding all ranges...")
print(f"Fetching: {main_url}")

try:
    response = requests.get(main_url, headers=headers, verify=False, timeout=15)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all range links (links with range= parameter)
    range_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        if 'range=' in href:
            if href.startswith('/'):
                href = base_url + href
            range_links.append(href)

    # Remove duplicates
    range_links = list(set(range_links))

    print(f"✓ Found {len(range_links)} ranges to scrape")

    # Show first 10 ranges
    print("\nFirst 10 ranges:")
    for i, link in enumerate(range_links[:10], 1):
        range_part = link.split('range=')[1] if 'range=' in link else ''
        print(f"  {i:2d}. {range_part}")

    if len(range_links) > 10:
        print(f"  ... and {len(range_links) - 10} more ranges")

    # Step 2: Scrape words from each range
    print(f"\nStep 2: Scraping words from all ranges...")
    print("=" * 60)

    all_words = []

    for i, range_url in enumerate(range_links, 1):
        try:
            range_name = range_url.split('range=')[1] if 'range=' in range_url else f'range_{i}'
            print(f"\n[{i}/{len(range_links)}] Scraping: {range_name}", end=" ... ")

            response = requests.get(range_url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract all words from first <td> in each <tr>
            words_in_range = []
            for tr in soup.find_all('tr'):
                tds = tr.find_all('td')
                if len(tds) >= 3:  # Only process rows with 3 td elements
                    first_td = tds[0]
                    # Find ALL links in the first td
                    links = first_td.find_all('a', href=True)
                    for link in links:
                        href = link.get('href', '')
                        if href.startswith('/bins/teny2/'):
                            word = link.get_text(strip=True)
                            # Remove leading special characters (-, ., etc.)
                            word = re.sub(r'^[^a-zA-Z]+', '', word)
                            # Skip words that start with a number or are empty
                            if word and not word[0].isdigit():
                                words_in_range.append(word)

            all_words.extend(words_in_range)
            print(f"✓ {len(words_in_range)} words")

            # Be polite to the server
            time.sleep(0.5)

        except Exception as e:
            print(f"✗ Error: {e}")
            continue

    # Remove duplicates and sort alphabetically
    all_words = sorted(set(all_words), key=lambda x: x.lower())

    print("\n" + "=" * 60)
    print(f"✓ TOTAL: {len(all_words)} unique Malagasy words extracted")

except Exception as e:
    print(f"\n✗ Error: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 60)
print("SCRAPING COMPLETED")
print("=" * 60)

Scraping Complete Malagasy Dictionary

Step 1: Finding all ranges...
Fetching: https://tenymalagasy.org/bins/alphaLists?lang=mg
✓ Found 47 ranges to scrape

First 10 ranges:
   1. abe-akak
   2. l
   3. fim-fir
   4. b-c
   5. t-taf
   6. mana
   7. mam
   8. mao-miaz
   9. k
  10. tb-tn
  ... and 37 more ranges

Step 2: Scraping words from all ranges...

[1/47] Scraping: abe-akak ... ✓ 3320 words

[2/47] Scraping: l ... ✓ 3034 words

[3/47] Scraping: fim-fir ... ✓ 1880 words

[4/47] Scraping: b-c ... ✓ 2770 words

[5/47] Scraping: t-taf ... ✓ 2284 words

[6/47] Scraping: mana ... ✓ 4964 words

[7/47] Scraping: mam ... ✓ 5686 words

[8/47] Scraping: mao-miaz ... ✓ 2924 words

[9/47] Scraping: k ... ✓ 5060 words

[10/47] Scraping: tb-tn ... ✓ 1031 words

[11/47] Scraping: fife-fil ... ✓ 3361 words

[12/47] Scraping: i ... ✓ 9139 words

[13/47] Scraping: fifa ... ✓ 516 words

[14/47] Scraping: akal-am ... ✓ 5015 words

[15/47] Scraping: fana ... ✓ 3764 words

[16/47] Scraping: an ... ✓ 8

In [117]:
# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print("=" * 60)
print("Scraping Complete Malagasy Dictionary")
print("=" * 60)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

# Step 1: Get all range links and letter links
base_url = "https://tenymalagasy.org"
patro_base_url = "https://tenymalagasy.org/bins/patroLists?l="

print(f"\nStep 1: Finding all ranges and letters...")
print(f"Fetching alphaLists: {patro_base_url}")

try:
    response = requests.get(patro_base_url, headers=headers, verify=False, timeout=15)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all range links (links with range= parameter)
    range_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        if 'range=' in href:
            if href.startswith('/'):
                href = base_url + href
            range_links.append(href)

    # Remove duplicates
    range_links = list(set(range_links))

    print(f"✓ Found {len(range_links)} alphaList ranges")

    # Generate patroList links for all letters A-Z
    patro_links = []
    for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
        patro_links.append(patro_base_url + letter)

    print(f"✓ Generated {len(patro_links)} patroList pages")

    # Combine all links
    all_links = range_links + patro_links

    print(f"✓ Total pages to scrape: {len(all_links)}")

    # Show samples
    print("\nSample alphaList ranges:")
    for i, link in enumerate(range_links[:5], 1):
        range_part = link.split('range=')[1] if 'range=' in link else ''
        print(f"  {i}. {range_part}")

    print("\nSample patroList letters:")
    for i, link in enumerate(patro_links[:5], 1):
        letter = link.split('l=')[1] if 'l=' in link else ''
        print(f"  {i}. {letter}")

    # Step 2: Scrape words from all pages
    print(f"\nStep 2: Scraping words from all pages...")
    print("=" * 60)

    all_words = []

    for i, url in enumerate(patro_links, 1):
        try:
            # Determine type
            if 'range=' in url:
                page_name = url.split('range=')[1]
                page_type = "alphaList"
            else:
                page_name = url.split('l=')[1] if 'l=' in url else f'page_{i}'
                page_type = "patroList"

            print(f"\n[{i}/{len(patro_links)}] {page_type}: {page_name}", end=" ... ")

            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract all words from first <td> in each <tr>
            words_in_range = []
            for tr in soup.find_all('tr'):
                tds = tr.find_all('td')
                if len(tds) >= 3:  # Only process rows with 3 td elements
                    first_td = tds[0]
                    # Find ALL links in the first td
                    links = first_td.find_all('a', href=True)
                    for link in links:
                        href = link.get('href', '')
                        if href.startswith('/bins/teny2/'):
                            word = link.get_text(strip=True)
                            # Remove leading special characters (-, ., etc.)
                            word = re.sub(r'^[^a-zA-Z]+', '', word)
                            # Skip words that start with a number or are empty
                            if word and not word[0].isdigit():
                                words_in_range.append(word)

            all_words.extend(words_in_range)
            print(f"✓ {len(words_in_range)} words")

            # Be polite to the server
            time.sleep(0.5)

        except Exception as e:
            print(f"✗ Error: {e}")
            continue

    # Remove duplicates and sort alphabetically
    all_words = sorted(set(all_words), key=lambda x: x.lower())

    print("\n" + "=" * 60)
    print(f"✓ TOTAL: {len(all_words)} unique Malagasy words extracted")

except Exception as e:
    print(f"\n✗ Error: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 60)
print("SCRAPING COMPLETED")
print("=" * 60)

Scraping Complete Malagasy Dictionary

Step 1: Finding all ranges and letters...
Fetching alphaLists: https://tenymalagasy.org/bins/patroLists?l=
✓ Found 0 alphaList ranges
✓ Generated 26 patroList pages
✓ Total pages to scrape: 26

Sample alphaList ranges:

Sample patroList letters:
  1. A
  2. B
  3. C
  4. D
  5. E

Step 2: Scraping words from all pages...

[1/26] patroList: A ... ✓ 34 words

[2/26] patroList: B ... ✓ 10 words

[3/26] patroList: C ... ✓ 6 words

[4/26] patroList: D ... ✓ 5 words

[5/26] patroList: E ... ✓ 7 words

[6/26] patroList: F ... ✓ 8 words

[7/26] patroList: G ... ✓ 5 words

[8/26] patroList: H ... ✓ 4 words

[9/26] patroList: I ... ✓ 10 words

[10/26] patroList: J ... ✓ 7 words

[11/26] patroList: K ... ✓ 3 words

[12/26] patroList: L ... ✓ 4 words

[13/26] patroList: M ... ✓ 10 words

[14/26] patroList: N ... ✓ 5 words

[15/26] patroList: O ... ✓ 2 words

[16/26] patroList: P ... ✓ 3 words

[17/26] patroList: Q ... ✓ 0 words

[18/26] patroList: R ... ✓ 112

In [1]:
# === ARRÊTER LE SERVEUR EXISTANT ===
import os
import signal
import subprocess

def stop_existing_servers():
    """Arrête tous les serveurs sur le port 8000"""
    print("🛑 Arrêt des serveurs existants...")

    try:
        # Trouver les processus sur le port 8000
        result = subprocess.run(
            ["fuser", "-k", "8000/tcp"],
            capture_output=True,
            text=True
        )
        print("✅ Port 8000 libéré")
    except:
        print("ℹ️  Aucun processus sur le port 8000")

    try:
        # Arrêter ngrok
        ngrok.disconnect()
        ngrok.kill()
        print("✅ Ngrok arrêté")
    except:
        print("ℹ️  Ngrok déjà arrêté")

    print("✅ Nettoyage terminé")

# Exécuter la fonction
stop_existing_servers()

🛑 Arrêt des serveurs existants...
✅ Port 8000 libéré
ℹ️  Ngrok déjà arrêté
✅ Nettoyage terminé


# Test de mot le plus semblable (Autocorrect)

In [2]:
from rapidfuzz import process, fuzz

# Load dictionary
with open('malagasy_complete_dictionary.txt', 'r', encoding='utf-8') as f:
    dictionary = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(dictionary):,} words\n")

# Test word
test_word = "Cedy"  # Change this to your word

# Find best 5 matches
best_matches = process.extract(test_word, dictionary, scorer=fuzz.ratio, limit=5)

print(f"Input: {test_word}\n")
print("Top 5 matches:")
for i, (word, score, idx) in enumerate(best_matches, 1):
    print(f"{i}. {word} - {score:.1f}%")

Loaded 107,116 words

Input: Cedy

Top 5 matches:
1. edy - 85.7%
2. bedy - 75.0%
3. endy - 75.0%
4. kedy - 75.0%
5. sedy - 75.0%


# Suggestion d'autocomplétion

In [17]:
from rapidfuzz import process, fuzz

# Load dictionary
with open('malagasy_complete_dictionary.txt', 'r', encoding='utf-8') as f:
    dictionary = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(dictionary):,} words\n")

# Choose your word here
word = "Randria"  # Change this to test different words

# Autocomplete suggestions
prefix_matches = [w for w in dictionary if w.lower().startswith(word.lower())]
suggestions = sorted(prefix_matches, key=lambda x: (len(x), x.lower()))[:5]

print(f"Word: '{word}'")
print(f"\nAutocomplete suggestions:")
for i, s in enumerate(suggestions, 1):
    print(f"  {i}. {s}")

# Next word prediction (if word is complete)
if word in dictionary:
    idx = dictionary.index(word)
    next_words = dictionary[idx+1:idx+6]

    print(f"\nNext word predictions:")
    for i, w in enumerate(next_words, 1):
        print(f"  {i}. {w}")

Loaded 107,116 words

Word: 'Randria'

Autocomplete suggestions:
  1. randria
  2. randriana
  3. randriatra
  4. randriamparany


# Contexte d'un mot

In [15]:
import requests
import time

def get_article_text(title):
    url = "https://mg.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "formatversion": 2,
        "prop": "extracts",
        "explaintext": True,
        "redirects": 1,
        "titles": title
    }

    headers = {
        "User-Agent": "MalagasyNLPResearch/1.0 (contact: youremail@example.com)"
    }

    r = requests.get(url, params=params, headers=headers, timeout=15)

    # ---- SAFETY CHECKS ----
    if r.status_code != 200:
        print("HTTP ERROR:", r.status_code)
        return ""

    if "application/json" not in r.headers.get("Content-Type", ""):
        print("NOT JSON RESPONSE")
        print(r.text[:200])
        return ""

    data = r.json()

    try:
        page = data["query"]["pages"][0]
        return page.get("extract", "")
    except Exception:
        return ""


In [16]:
text = get_article_text("Antsirabe")

if text:
    # Split text into sentences using regex
    sentences = re.split(r'(?<=[.!?]) +', text)
    # Get the first 10 sentences
    first_10_sentences = ' '.join(sentences[:5])
    print(first_10_sentences)
else:
    print("No content found")

19.865860°S 47.033330°E﻿ / -19.865860; 47.033330

Antsirabe dia tanàna fahatelo lehibe ao Madagasikara raha ny isan'ny mponina no jerena.


== Jeografia ==
Eo amin'ny afovoan'ny nosy no misy azy, 168 km atsimon'Antananarivo manaraka ny lalam-pirenena faha-7. Renivohitry ny faritr'i Vakinankaratra izy. Araka ny fanisana natao ny Janoary 2005 dia manodidina ny 182 804 ireo mponina ao.
Amin'ny haavo (altitude) 1 500 metatra Antsirabe ka voahodidina "volcan" izy. Antsirabe no tanàna mangatsiaka indrindra ao Madagasikara ka tonga hatrany amin'ny 0°C rehefa tonga ny vanim-potoana ririnina.
Manana seranam-piaramanidina Antsirabe.


== Tantara ==
Antsirabe dia midika hoe "ilay feno sira".Io anarana io dia avy amin'ny nahitana ilay rano misy sira. Ilay tanàna dia namboarin'ilay misionera norvezianina tamin'ny faran'ny taonjato faha-19, izay, angamba nahita ilay fahagagana izay tsy takatrin' ny saina, izany hoe, ilay rano tany Antsirabe dia anisan'ny rano mahery eran-tany.


# Lemmanization

In [8]:

def load_words(filename="malagasy_complete_dictionary.txt"):
    with open(filename, "r", encoding="utf-8") as f:
        return [line.strip().lower() for line in f if line.strip()]


def levenshtein(a, b):
    if a == b:
        return 0
    if len(a) == 0:
        return len(b)
    if len(b) == 0:
        return len(a)

    dp = [[0] * (len(b) + 1) for _ in range(len(a) + 1)]

    for i in range(len(a) + 1):
        dp[i][0] = i
    for j in range(len(b) + 1):
        dp[0][j] = j

    for i in range(1, len(a) + 1):
        for j in range(1, len(b) + 1):
            cost = 0 if a[i - 1] == b[j - 1] else 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,      # suppression
                dp[i][j - 1] + 1,      # insertion
                dp[i - 1][j - 1] + cost  # substitution
            )
    return dp[-1][-1]


# -----------------------------
# Lemmatisation malagasy
# -----------------------------
PREFIXES = ["mpam", "mpan", "maha", "mam", "man", "fan", "fam", "fi", "ma", "mi"]
SUFFIXES = ["ana", "ina", "na"]

def lemmatize(word):
    w = word.lower()

    # enlever préfixe (le plus long d'abord)
    for p in sorted(PREFIXES, key=len, reverse=True):
        if w.startswith(p):
            w = w[len(p):]
            break

    # enlever suffixe
    for s in sorted(SUFFIXES, key=len, reverse=True):
        if w.endswith(s):
            w = w[:-len(s)]
            break

    return w


# -----------------------------
# Comparaison Levenshtein
# -----------------------------
def find_closest_words(target_word, words, top_n=10):
    target_root = lemmatize(target_word)

    results = []
    for w in words:
        root = lemmatize(w)
        dist = levenshtein(target_root, root)
        results.append((w, root, dist))

    results.sort(key=lambda x: x[2])
    print(results)
    return results[:top_n]


# -----------------------------
# Exemple d'utilisation
# -----------------------------
if __name__ == "__main__":
    words = load_words("fototeny.txt")

    target = "misongona"
    target_root = lemmatize(target)

    print(f"\nMot : {target}")
    print(f"Lemme estimé : {target_root}\n")

    closest = find_closest_words(target, words, top_n=10)

    print("Mots les plus proches (Levenshtein) :")
    for word, root, dist in closest:
        print(f"  {word:<20} → {root:<10} (distance={dist})")



Mot : misongona
Lemme estimé : songo

[('songona', 'songo', 0), ('bongo', 'bongo', 1), ('dongo', 'dongo', 1), ('fongo', 'fongo', 1), ('kongona', 'kongo', 1), ('longo', 'longo', 1), ('mongo', 'mongo', 1), ('rongo', 'rongo', 1), ('songa', 'songa', 1), ('sonjo', 'sonjo', 1), ('tsongo', 'tsongo', 1), ('vongo', 'vongo', 1), ('ango', 'ango', 2), ('angona', 'ango', 2), ('bango', 'bango', 2), ('bangoana', 'bango', 2), ('bingo', 'bingo', 2), ('bonga', 'bonga', 2), ('bongina', 'bong', 2), ('dango', 'dango', 2), ('dondona', 'dondo', 2), ('donga', 'donga', 2), ('dongy', 'dongy', 2), ('donto', 'donto', 2), ('fonga', 'fonga', 2), ('fongana', 'fong', 2), ('fonjo', 'fonjo', 2), ('fono', 'fono', 2), ('goangoana', 'goango', 2), ('gogo', 'gogo', 2), ('gonagona', 'gonago', 2), ('hogo', 'hogo', 2), ('honkona', 'honko', 2), ('hono', 'hono', 2), ('jongy', 'jongy', 2), ('jonjona', 'jonjo', 2), ('jono', 'jono', 2), ('konga', 'konga', 2), ('konjo', 'konjo', 2), ('konko', 'konko', 2), ('lango', 'lango', 2), ('l

# Lancer le serveur ngrok

In [10]:
# === CELLULE 1: Installation ===
!pip install fastapi uvicorn nest-asyncio scikit-learn pandas numpy --quiet pyngrok

In [11]:
!ngrok config add-authtoken 2KoG2t9MfM0zljJS7Bk9IP8Vczb_6kYMafu5VF7LxL2H3uMcY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
from pydantic import BaseModel
from typing import List

class WordCheckRequest(BaseModel):
    word: str
    top_k: int = 5  # number of matches to return, default

In [13]:
import numpy as np
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional, Dict, Any
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import threading
import time
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# 🔥 APPLIQUER nest_asyncio
nest_asyncio.apply()

print("✅ nest_asyncio appliqué")
# ==================== FONCTION DE PRÉDICTION CORRIGÉE ====================

# ==================== MODÈLES PYDANTIC ====================
class PredictionRequest(BaseModel):
    features: List[float]

class MT5DataRequest(BaseModel):
    closes: List[float]
    returns: Optional[List[float]] = None

# ==================== INITIALISATION FASTAPI ====================
app = FastAPI(
    title="🎯 BNN Trading API avec Stratégie Two-Tier",
    description="API de prédiction avec modèle BNN et stratégie two-tier avancée",
    version="3.0.0"
)

# CORS - Autoriser toutes les origines (pour ngrok)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ==================== ROUTES CORRIGÉES ====================
@app.get("/")
async def root():
    return {
        "message": "Test si miova le API",
        "status": "online",
    }

@app.post("/check_word")
async def check_word(request: WordCheckRequest):
    if not request.word:
        return {"error": "No word provided"}

    # Find best matches
    matches = process.extract(request.word, dictionary, scorer=fuzz.ratio, limit=request.top_k)

    # Format output
    result = [{"word": word, "score": score} for word, score, idx in matches]
    return {
        "input": request.word,
        "top_matches": result
    }

@app.post("/autocorrect")
async def autocorrect(request: WordCheckRequest):
    word = request.word.strip()
    top_k = request.top_k

    if not word:
        return {"error": "No word provided"}

    # Autocomplete suggestions
    prefix_matches = [w for w in dictionary if w.lower().startswith(word.lower())]
    suggestions = sorted(prefix_matches, key=lambda x: (len(x), x.lower()))[:top_k]

    # Next word prediction (if word exists in dictionary)
    next_words = []
    if word in dictionary:
        idx = dictionary.index(word)
        next_words = dictionary[idx+1:idx+1+top_k]

    return {
        "input_word": word,
        "autocomplete_suggestions": suggestions,
        "next_word_predictions": next_words
    }

@app.post("/lemanization")
async def autocorrect(request: WordCheckRequest):
    word = request.word.strip()
    top_k = request.top_k

    closest = find_closest_words(word, dictionary, top_n=10)
    # Next word prediction (if word exists in dictionary)
    next_words = []
    if word in dictionary:
        idx = dictionary.index(word)
        next_words = dictionary[idx+1:idx+1+top_k]

    return {
        "input_word": word,
        "Fototeny": closest,
    }


✅ nest_asyncio appliqué


In [14]:
# === CELLULE 3: Démarrage ===
import threading
from fastapi import FastAPI
import uvicorn


def start_fastapi():
    """Démarre le serveur FastAPI"""
    print("🚀 Starting FastAPI server...")

    # Configuration ngrok
    print("🔗 Setting up ngrok tunnel...")
    tunnel = ngrok.connect(8000)
    global public_url
    public_url = tunnel.public_url

    print("\n" + "="*60)
    print("🎯 BNN TRADING API - FASTAPI")
    print("="*60)
    print(f"🌐 PUBLIC URL: {public_url}")
    print(f"📚 DOCS: {public_url}/docs")
    print(f"🩺 HEALTH: {public_url}/health")
    print("\n⚡ Ready for predictions!")
    print("="*60)

    # Démarrer le serveur
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Démarrer dans un thread
import threading
server_thread = threading.Thread(target=start_fastapi, daemon=True)
server_thread.start()

# Attendre le démarrage
import time
time.sleep(3)
print("⏳ Server is starting... (check output above for URL)")

🚀 Starting FastAPI server...
🔗 Setting up ngrok tunnel...

🎯 BNN TRADING API - FASTAPI
🌐 PUBLIC URL: https://c7afd49aa549.ngrok-free.app
📚 DOCS: https://c7afd49aa549.ngrok-free.app/docs
🩺 HEALTH: https://c7afd49aa549.ngrok-free.app/health

⚡ Ready for predictions!


INFO:     Started server process [94130]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


⏳ Server is starting... (check output above for URL)
